In [1]:
import os
import random

import numpy as np
import ROOT as r
from matplotlib import pyplot as plt

# ---------------------------------------------------------------------------
# Import ROOT libraries for MadGraph generated rootfiles.
# ---------------------------------------------------------------------------
r.gROOT.ProcessLine('.include /Users/isaac/Work/MG5_aMC_v2_6_7')
r.gInterpreter.Declare(
    '#include "/Users/isaac/Work/Delphes-3.4.2/external/ExRootAnalysis/ExRootTreeReader.h"')
r.gInterpreter.Declare(
    '#include "/Users/isaac/Work/MG5_aMC_v2_6_7/ExRootAnalysis/ExRootAnalysis/ExRootClasses.h"'
)
r.gInterpreter.Declare(
    '#include "/Users/isaac/Work/MG5_aMC_v2_6_7/ExRootAnalysis/ExRootAnalysis/ExRootLHEFReader.h"'
)
r.gSystem.Load(
    "/Users/isaac/Work/MG5_aMC_v2_6_7/ExRootAnalysis/libExRootAnalysis.so")

# ---------------------------------------------------------------------------
# Set up environment
# ---------------------------------------------------------------------------
os.environ['TERM'] = 'linux'
random.seed(1)

Welcome to JupyROOT 6.22/00


In [3]:
signal_chain = r.TChain("LHEF")
bg_chain=r.TChain("LHEF")
signal_chain.Add(
    "~/Work/MuonE/2to3sc/Events/nocut_0.1GeV/unweighted_events.root")
bg_chain.Add("~/Work/MuonE/SMmuegamma/Events/pt4_0.001_pta_0.001/unweighted_events.root")

1

In [4]:
def T(theta_e):
    """T: defined as $E_4 - m_e$."""
    t = 2 * 0.000511 / (((150 + 0.000511)**2) /
                        ((np.cos(theta_e / 1000)**2) * (150**2 - 0.1057**2))
                        - 1)
    return t


def theta_mu_theory(theta_e):
    """The expression for the theta_mu vs theta_e relation in SM 2 to 2
    QED process"""
    theta = 1000 * np.arctan((np.sqrt(T(theta_e) * (2 * 0.000511 + T(theta_e)))
                              * np.sin(theta_e / 1000)) /
                             (np.sqrt(150**2 - 0.1057**2)
                              - np.sqrt(T(theta_e) * (2 * 0.000511 + T(theta_e))) * np.cos(theta_e / 1000)))
    return theta


# Generate variable and prepare for the plots.
theta_mu2 = np.vectorize(theta_mu_theory)
theta_e_variable = np.linspace(0, 30, 500)
mu_variable = theta_mu2(theta_e_variable)

In [8]:
luminosity = 1.5e+04  # Luminosity, which is 1.5 \times 10^4 pb^-1
signal_x_section = 248180.0  # Signal event cross section
bg_x_section = 1500450770.0
signal_above_curve_number = 0
bg_above_curve_number = 0
signal_theta_e_list = []
signal_theta_mu_list = []
bg_theta_e_list = []
bg_theta_mu_list = []

In [9]:
for event in signal_chain:
    for particle in event.Particle:
            # Pick out final state electron
            if(particle.Status == 1 and particle.PID == 11):
                electron_vector = r.TLorentzVector()  # electron 4-vector
                pt4 = particle.PT  # electron pt
                electron_vector.SetPtEtaPhiM(
                    particle.PT, particle.Eta, particle.Phi, particle.M)
                theta_e = electron_vector.Theta() * 1000  # theta_e
            # Pick out final state muon
            elif(particle.Status == 1 and particle.PID == 13):
                muon_vector = r.TLorentzVector()  # muon 4-vector
                muon_vector.SetPtEtaPhiM(
                    particle.PT, particle.Eta, particle.Phi, particle.M)
                theta_mu = muon_vector.Theta() * 1000  # theta_mu
    
    if(pt4 > 0.001 and theta_e <= 30):
        signal_theta_e_list.append(theta_e)
        signal_theta_mu_list.append(theta_mu)
        if(theta_mu > theta_mu_theory(theta_e)):
            signal_above_curve_number += 1

In [15]:
signal_efficiency = float(signal_above_curve_number) / 10000.0
signal_rescaled_number = luminosity * signal_x_section * signal_efficiency
print(signal_above_curve_number, signal_efficiency, signal_rescaled_number)

(2661, 0.2661, 990610470.0)
